In [6]:
# needs to use the superraenn conda environ because of 
# version-specific dependencies with python
# (which is annoying)
import numpy as np
from matplotlib import pyplot as plt
import urllib
import os
import sys
from astropy.io import fits
from sklearn.model_selection import train_test_split
import pandas as pd
#import GCRCatalogs
from astropy.cosmology import Planck15 as P15
from astropy import units as u
import time 
import seaborn as sns
import pandas as pd
%matplotlib inline
import qp

In [7]:
sns.set_context("talk",font_scale=1.5)

sns.set_style('white', {'axes.linewidth': 0.5})
plt.rcParams['xtick.major.size'] = 15
plt.rcParams['ytick.major.size'] = 15

plt.rcParams['xtick.minor.size'] = 10
plt.rcParams['ytick.minor.size'] = 10
plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.minor.width'] = 2

plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['ytick.right'] = True

plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

In [1]:
dirpath = "/global/cfs/cdirs/desc-td/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/zquants/"

import glob

In [35]:
SNIa_quants = glob.glob(dirpath + "/*SNIa_GHOST.fits")

In [75]:
for fn in SNIa_quants:
    ens_quant = qp.read(fn)
    bins = ens_quant.metadata()['quants']
    pdfs = ens_quant.pdf(bins)
    bins = bins[0][1:-1]
    pdfs = pdfs[:, 1:-1]
    p50 = ens_quant.ancil['logp50']
    break

In [87]:
varnames = []
df_dict = {}
for i in np.arange(9):
    qpidx = (i+1)*10
    varnames.append("HOSTGAL_ZPHOT_Q%i"%qpidx)        
    df_dict[varnames[i]] = pdfs[:, i]
df_dict['LOGP50'] = p50
df_dict['GALID'] = ens_quant.ancil['GALID']
df = pd.DataFrame(df_dict)

In [89]:
df

,ZPHOT_Q10,ZPHOT_Q20,ZPHOT_Q30,ZPHOT_Q40,ZPHOT_Q50,ZPHOT_Q60,ZPHOT_Q70,ZPHOT_Q80,ZPHOT_Q90,LOGP50,GALID
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,2.788924,10315846307
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,2.566957,10315583641
2,0.0,0.0,13.415967,0.0,0.0,0.000000,0.000000,0.0,0.0,3.291289,10312559336
3,0.0,0.0,0.000000,0.0,0.0,0.000000,13.606328,0.0,0.0,3.142981,10313381246
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.371636,10313261803
...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.062102,10314199158
996,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.542279,10314689139
997,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.042065,10316518574
998,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.131964,10313671757


In [90]:
df.to_csv(dirpath + "/TESTQUANTILES_SNII.HOSTLIB", sep=' ', index=False)

In [92]:
SNII_HOSTLIB = pd.read_csv("/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/noheader/SNII_GHOST_PHOTOZ.HOSTLIB", delim_whitespace=True)
SNIa_HOSTLIB = pd.read_csv("/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/noheader/SNIa_GHOST_PHOTOZ.HOSTLIB", delim_whitespace=True)
SNIbc_HOSTLIB = pd.read_csv("/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/noheader/SNIbc_GHOST_PHOTOZ.HOSTLIB", delim_whitespace=True)
UNMATCHED_HOSTLIB = pd.read_csv("/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/noheader/UNMATCHED_COSMODC2_PHOTOZ.HOSTLIB", delim_whitespace=True)
UNMATCHED_KN_HOSTLIB = pd.read_csv("/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/noheader/UNMATCHED_KN_COSMODC2.HOSTLIB", delim_whitespace=True)

In [ ]:
#doing things another way for now -- just getting quants from a normal distribution
#centered on the true redshift with scale z=0.5 (probably way too high but as a first pass)

from scipy.stats import norm
bins = np.linspace(0, 1, 11)[1:-1]

varnames = []
pdf_dict = {}

pdfs = [[],[],[],[],[],[],[],[],[]]
for j in np.arange(9):
    qpidx = (j+1)*10
    varnames.append("HOSTGAL_ZPHOT_Q%i"%qpidx)      
    pdf_dict[varnames[j]] = []
        
for i in np.arange(len(SNII_HOSTLIB)):
    pdf_vals = norm(loc= SNII_HOSTLIB['ZTRUE'][i], scale=0.5).ppf(bins)
    for j in np.arange(len(pdf_vals)):
        qpidx = (j+1)*10
        varnames.append("HOSTGAL_ZPHOT_Q%i"%qpidx)      
        pdf_dict[varnames[j]].append(pdf_vals[j])

In [133]:
df_hokey = pd.DataFrame(pdf_dict)
df_hokey['GALID'] = SNII_HOSTLIB['GALID'].values

df.to_csv(dirpath + "/TESTQUANTILES_SNII.HOSTLIB", sep=' ', index=False)

['HOSTGAL_ZPHOT_Q10']